# Airfoil's Aerodynamic Coefficient Prediction using ANNs

In [1]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

In [2]:
# importing the dependencies
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow import keras
from keras import Input
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.metrics import RootMeanSquaredError
from keras.callbacks import ReduceLROnPlateau
import matplotlib.pyplot as plt

In [3]:
# reading the csv file
df4 = pd.read_csv("/home/nevilcp/ML_Aero/results/NACA4D_10/NACA4D_10.csv")
df5 = pd.read_csv("/home/nevilcp/ML_Aero/results/NACA5D_10/NACA5D_10.csv")

In [4]:
# printing the first 10 rows of the dataset of NACA4D_10
df4.head(10)

,t,m,p,yU1,yU2,yU3,yU4,yU5,yU6,yU7,...,yL7,yL8,yL9,yL10,alpha,M,Re,CL,CD,Cm
0,5,0,5,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,...,-0.014949,-0.009639,-0.004948,-0.001694,-10.0,0.2,100000,-1.158,0.02657,0.005
1,5,0,5,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,...,-0.014949,-0.009639,-0.004948,-0.001694,-9.0,0.2,100000,-1.043,0.02194,0.004
2,5,0,5,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,...,-0.014949,-0.009639,-0.004948,-0.001694,-8.0,0.2,100000,-0.928,0.01876,0.004
3,5,0,5,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,...,-0.014949,-0.009639,-0.004948,-0.001694,-7.0,0.2,100000,-0.813,0.01463,0.004
4,5,0,5,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,...,-0.014949,-0.009639,-0.004948,-0.001694,-6.0,0.2,100000,-0.697,0.02806,0.003
5,5,0,5,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,...,-0.014949,-0.009639,-0.004948,-0.001694,-5.0,0.2,100000,-0.581,0.01660,0.003
6,5,0,5,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,...,-0.014949,-0.009639,-0.004948,-0.001694,-4.0,0.2,100000,-0.465,0.01146,0.002
7,5,0,5,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,...,-0.014949,-0.009639,-0.004948,-0.001694,-3.0,0.2,100000,-0.349,0.00893,0.002
8,5,0,5,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,...,-0.014949,-0.009639,-0.004948,-0.001694,-2.0,0.2,100000,-0.233,0.01393,0.001
9,5,0,5,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,...,-0.014949,-0.009639,-0.004948,-0.001694,-1.0,0.2,100000,-0.116,0.01358,0.001


In [5]:
# printing the first 10 rows of the dataset of NACA5D_10
df5.head(10)

,t,CL_design,p,s,yU1,yU2,yU3,yU4,yU5,yU6,...,yL7,yL8,yL9,yL10,alpha,M,Re,CL,CD,Cm
0,5,0.0,5,0,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,...,-0.014949,-0.009639,-0.004948,-0.001694,-10.0,0.1,100000,-1.140,0.02494,0.005
1,5,0.0,5,0,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,...,-0.014949,-0.009639,-0.004948,-0.001694,-9.0,0.1,100000,-1.027,0.02116,0.004
2,5,0.0,5,0,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,...,-0.014949,-0.009639,-0.004948,-0.001694,-8.0,0.1,100000,-0.914,0.01697,0.004
3,5,0.0,5,0,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,...,-0.014949,-0.009639,-0.004948,-0.001694,-7.0,0.1,100000,-0.800,0.01364,0.003
4,5,0.0,5,0,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,...,-0.014949,-0.009639,-0.004948,-0.001694,-6.0,0.1,100000,-0.687,0.02483,0.003
5,5,0.0,5,0,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,...,-0.014949,-0.009639,-0.004948,-0.001694,-5.0,0.1,100000,-0.572,0.01565,0.002
6,5,0.0,5,0,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,...,-0.014949,-0.009639,-0.004948,-0.001694,-4.0,0.1,100000,-0.458,0.01114,0.002
7,5,0.0,5,0,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,...,-0.014949,-0.009639,-0.004948,-0.001694,-3.0,0.1,100000,-0.344,0.00882,0.001
8,5,0.0,5,0,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,...,-0.014949,-0.009639,-0.004948,-0.001694,-2.0,0.1,100000,-0.229,0.01391,0.001
9,5,0.0,5,0,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,...,-0.014949,-0.009639,-0.004948,-0.001694,-1.0,0.1,100000,-0.115,0.01357,0.000


In [6]:
# checking for missing values in the NACA4D_10 dataset
df4.isnull().sum()

t        0
m        0
p        0
yU1      0
yU2      0
yU3      0
yU4      0
yU5      0
yU6      0
yU7      0
yU8      0
yU9      0
yU10     0
yL1      0
yL2      0
yL3      0
yL4      0
yL5      0
yL6      0
yL7      0
yL8      0
yL9      0
yL10     0
alpha    0
M        0
Re       0
CL       0
CD       0
Cm       0
dtype: int64

In [7]:
# checking for missing values in the NACA5D_10 dataset
df5.isnull().sum()

t            0
CL_design    0
p            0
s            0
yU1          0
yU2          0
yU3          0
yU4          0
yU5          0
yU6          0
yU7          0
yU8          0
yU9          0
yU10         0
yL1          0
yL2          0
yL3          0
yL4          0
yL5          0
yL6          0
yL7          0
yL8          0
yL9          0
yL10         0
alpha        0
M            0
Re           0
CL           0
CD           0
Cm           0
dtype: int64

In [8]:
# description of the NACA4D_10 dataset
df4.describe()

,t,m,p,yU1,yU2,yU3,yU4,yU5,yU6,yU7,...,yL7,yL8,yL9,yL10,alpha,M,Re,CL,CD,Cm
count,176400.000000,176400.000000,176400.000000,176400.000000,176400.000000,176400.000000,176400.000000,176400.000000,176400.000000,176400.000000,...,176400.000000,176400.000000,176400.000000,176400.000000,176400.000000,176400.00000,176400.000000,176400.000000,176400.000000,176400.000000
mean,20.000000,4.500000,40.000000,0.056883,0.096159,0.125194,0.138691,0.136083,0.120107,0.094865,...,-0.025367,-0.013472,-0.006526,-0.003090,0.000000,0.20000,300000.000000,0.692283,0.026646,-0.134151
std,10.000028,2.872289,22.912943,0.030652,0.043042,0.052297,0.056240,0.054535,0.048225,0.039002,...,0.037872,0.026573,0.014169,0.004346,6.055318,0.08165,141421.757093,0.926925,0.010960,0.113007
min,5.000000,0.000000,5.000000,0.009825,0.017881,0.023099,0.024990,0.023669,0.019965,0.014949,...,-0.104640,-0.067474,-0.034638,-0.011856,-10.000000,0.10000,100000.000000,-1.489000,0.001800,-0.514000
25%,10.000000,2.000000,22.500000,0.031423,0.060652,0.084089,0.094866,0.094677,0.083039,0.065317,...,-0.053389,-0.032844,-0.016811,-0.006339,-5.000000,0.10000,200000.000000,-0.011000,0.018990,-0.199000
50%,20.000000,4.500000,40.000000,0.054270,0.096111,0.124907,0.138240,0.136379,0.119788,0.094447,...,-0.025009,-0.014037,-0.006982,-0.003241,-0.000000,0.20000,300000.000000,0.691000,0.025390,-0.106000
75%,30.000000,7.000000,57.500000,0.078524,0.129458,0.166614,0.182079,0.176607,0.156303,0.123175,...,0.001474,0.004181,0.002635,-0.000235,5.000000,0.30000,400000.000000,1.379000,0.032540,-0.046000
max,35.000000,9.000000,75.000000,0.149810,0.208025,0.246146,0.264431,0.256034,0.230429,0.194578,...,0.074748,0.071348,0.042540,0.011816,10.000000,0.30000,500000.000000,3.599000,0.235470,0.025000


In [9]:
# description of the NACA5D_10 dataset
df5.describe()

,t,CL_design,p,s,yU1,yU2,yU3,yU4,yU5,yU6,...,yL7,yL8,yL9,yL10,alpha,M,Re,CL,CD,Cm
count,236670.000000,236670.000000,236670.000000,236670.000000,236670.000000,236670.000000,236670.000000,236670.000000,236670.000000,236670.000000,...,236670.000000,236670.000000,236670.000000,236670.000000,236670.000000,236670.000000,236670.000000,236670.000000,236670.000000,236670.000000
mean,19.964508,0.841171,27.648625,0.493345,0.061993,0.102135,0.127800,0.134156,0.122524,0.098694,...,-0.050760,-0.035252,-0.019158,-0.006666,0.000000,0.200000,300000.000000,0.219104,0.025913,0.008665
std,10.002176,0.595675,16.859596,0.499957,0.035497,0.050154,0.062040,0.067278,0.064190,0.054114,...,0.038246,0.024719,0.012716,0.004107,6.055314,0.081686,141421.655012,0.926011,0.011283,0.092562
min,5.000000,0.000000,5.000000,0.000000,0.002846,-0.006454,-0.022993,-0.039093,-0.048037,-0.046811,...,-0.171128,-0.127512,-0.072044,-0.023250,-10.000000,0.100000,100000.000000,-2.607000,0.002690,-0.169000
25%,10.000000,0.200000,15.000000,0.000000,0.033635,0.064809,0.081379,0.082846,0.074521,0.059894,...,-0.078123,-0.052613,-0.027989,-0.009654,-5.000000,0.100000,200000.000000,-0.485000,0.018260,-0.031000
50%,20.000000,0.800000,25.000000,0.000000,0.057851,0.101474,0.127721,0.133115,0.120174,0.099175,...,-0.050658,-0.034769,-0.018871,-0.006613,-0.000000,0.200000,300000.000000,0.218000,0.024650,-0.004000
75%,30.000000,1.400000,45.000000,1.000000,0.085249,0.134294,0.169672,0.179965,0.165684,0.137987,...,-0.022239,-0.016698,-0.009816,-0.003398,5.000000,0.300000,400000.000000,0.928000,0.031580,0.018000
max,35.000000,1.800000,75.000000,1.000000,0.165549,0.250220,0.311403,0.334353,0.308679,0.262679,...,0.048283,0.022775,0.009953,0.002097,10.000000,0.300000,500000.000000,3.020000,0.617020,0.453000


### Dropping columns

In [10]:
# dropping columns t, m & p in the NACA4D_10 dataset
df4 = df4.drop(columns=['t', 'm', 'p'], axis=1)
df4.head(10)

,yU1,yU2,yU3,yU4,yU5,yU6,yU7,yU8,yU9,yU10,...,yL7,yL8,yL9,yL10,alpha,M,Re,CL,CD,Cm
0,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,0.009639,0.004948,0.001694,...,-0.014949,-0.009639,-0.004948,-0.001694,-10.0,0.2,100000,-1.158,0.02657,0.005
1,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,0.009639,0.004948,0.001694,...,-0.014949,-0.009639,-0.004948,-0.001694,-9.0,0.2,100000,-1.043,0.02194,0.004
2,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,0.009639,0.004948,0.001694,...,-0.014949,-0.009639,-0.004948,-0.001694,-8.0,0.2,100000,-0.928,0.01876,0.004
3,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,0.009639,0.004948,0.001694,...,-0.014949,-0.009639,-0.004948,-0.001694,-7.0,0.2,100000,-0.813,0.01463,0.004
4,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,0.009639,0.004948,0.001694,...,-0.014949,-0.009639,-0.004948,-0.001694,-6.0,0.2,100000,-0.697,0.02806,0.003
5,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,0.009639,0.004948,0.001694,...,-0.014949,-0.009639,-0.004948,-0.001694,-5.0,0.2,100000,-0.581,0.01660,0.003
6,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,0.009639,0.004948,0.001694,...,-0.014949,-0.009639,-0.004948,-0.001694,-4.0,0.2,100000,-0.465,0.01146,0.002
7,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,0.009639,0.004948,0.001694,...,-0.014949,-0.009639,-0.004948,-0.001694,-3.0,0.2,100000,-0.349,0.00893,0.002
8,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,0.009639,0.004948,0.001694,...,-0.014949,-0.009639,-0.004948,-0.001694,-2.0,0.2,100000,-0.233,0.01393,0.001
9,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,0.009639,0.004948,0.001694,...,-0.014949,-0.009639,-0.004948,-0.001694,-1.0,0.2,100000,-0.116,0.01358,0.001


In [11]:
# dropping columns t, Cl_design, p & s in the NACA5D_10 dataset
df5 = df5.drop(columns=['t', 'CL_design', 'p', 's'], axis=1)
df5.head(10)

,yU1,yU2,yU3,yU4,yU5,yU6,yU7,yU8,yU9,yU10,...,yL7,yL8,yL9,yL10,alpha,M,Re,CL,CD,Cm
0,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,0.009639,0.004948,0.001694,...,-0.014949,-0.009639,-0.004948,-0.001694,-10.0,0.1,100000,-1.140,0.02494,0.005
1,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,0.009639,0.004948,0.001694,...,-0.014949,-0.009639,-0.004948,-0.001694,-9.0,0.1,100000,-1.027,0.02116,0.004
2,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,0.009639,0.004948,0.001694,...,-0.014949,-0.009639,-0.004948,-0.001694,-8.0,0.1,100000,-0.914,0.01697,0.004
3,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,0.009639,0.004948,0.001694,...,-0.014949,-0.009639,-0.004948,-0.001694,-7.0,0.1,100000,-0.800,0.01364,0.003
4,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,0.009639,0.004948,0.001694,...,-0.014949,-0.009639,-0.004948,-0.001694,-6.0,0.1,100000,-0.687,0.02483,0.003
5,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,0.009639,0.004948,0.001694,...,-0.014949,-0.009639,-0.004948,-0.001694,-5.0,0.1,100000,-0.572,0.01565,0.002
6,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,0.009639,0.004948,0.001694,...,-0.014949,-0.009639,-0.004948,-0.001694,-4.0,0.1,100000,-0.458,0.01114,0.002
7,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,0.009639,0.004948,0.001694,...,-0.014949,-0.009639,-0.004948,-0.001694,-3.0,0.1,100000,-0.344,0.00882,0.001
8,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,0.009639,0.004948,0.001694,...,-0.014949,-0.009639,-0.004948,-0.001694,-2.0,0.1,100000,-0.229,0.01391,0.001
9,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,0.009639,0.004948,0.001694,...,-0.014949,-0.009639,-0.004948,-0.001694,-1.0,0.1,100000,-0.115,0.01357,0.000


In [12]:
# shuffling all the rows for both the datasets
df4 = df4.sample(frac=1)

df5 = df5.sample(frac=1)

In [13]:
# defining the feature and target columns for both the datasets
X4 = df4.drop(columns=['CL', 'CD', 'Cm']) 
y4 = df4[['CL', 'CD', 'Cm']]

X5 = df5.drop(columns=['CL', 'CD', 'Cm'])
y5 = df5[['CL', 'CD', 'Cm']]

In [14]:
# displaying the features table for NACA4D_10
X4.head(10)

,yU1,yU2,yU3,yU4,yU5,yU6,yU7,yU8,yU9,yU10,...,yL4,yL5,yL6,yL7,yL8,yL9,yL10,alpha,M,Re
19046,0.062605,0.097748,0.124728,0.134321,0.127066,0.107294,0.080415,0.051865,0.026546,0.008950,...,-0.115592,-0.110034,-0.092903,-0.069531,-0.044842,-0.023083,-0.008006,10.0,0.2,200000
116332,0.083491,0.138419,0.180571,0.202920,0.201780,0.178295,0.139260,0.092728,0.047886,0.015152,...,-0.097893,-0.082627,-0.061860,-0.041143,-0.024334,-0.012409,-0.005296,3.0,0.1,500000
93273,0.045039,0.084886,0.115485,0.124815,0.118370,0.101230,0.076861,0.049876,0.025234,0.007849,...,-0.025134,-0.023796,-0.018867,-0.013192,-0.008303,-0.004649,-0.002344,2.0,0.2,200000
64947,0.045754,0.082059,0.109474,0.123481,0.123065,0.109808,0.086555,0.057813,0.029936,0.009570,...,-0.076683,-0.066340,-0.049870,-0.033075,-0.019484,-0.009786,-0.003975,5.0,0.1,300000
162156,0.090115,0.150842,0.181050,0.187625,0.176518,0.150285,0.113835,0.073778,0.037199,0.011398,...,-0.012279,-0.014156,-0.011446,-0.007852,-0.005104,-0.003362,-0.002374,5.0,0.2,200000
166348,0.066051,0.118708,0.163320,0.187445,0.183798,0.161071,0.124902,0.082419,0.041962,0.012731,...,-0.013053,-0.005943,0.000254,0.003712,0.003531,0.001234,-0.001126,-3.0,0.1,200000
50236,0.065504,0.114900,0.149250,0.163954,0.159391,0.139276,0.109471,0.073629,0.038546,0.012690,...,-0.136091,-0.124682,-0.100231,-0.069788,-0.042068,-0.020872,-0.007536,-6.0,0.3,300000
137968,0.051227,0.092923,0.127368,0.149340,0.156323,0.148549,0.128445,0.092597,0.049802,0.015248,...,-0.051808,-0.033636,-0.011218,0.008856,0.014061,0.008570,0.001045,9.0,0.1,500000
174770,0.027167,0.055856,0.084217,0.106966,0.120906,0.124691,0.119536,0.101195,0.059165,0.017814,...,0.005987,0.025719,0.044749,0.059800,0.061757,0.037253,0.009824,-2.0,0.3,300000
94278,0.090834,0.146324,0.186159,0.199824,0.190150,0.162714,0.123593,0.080506,0.041114,0.013241,...,-0.100084,-0.094702,-0.078153,-0.057081,-0.036246,-0.018804,-0.007095,-1.0,0.1,500000


In [15]:
# displaying the features table for NACA5D_10
X5.head(10)

,yU1,yU2,yU3,yU4,yU5,yU6,yU7,yU8,yU9,yU10,...,yL4,yL5,yL6,yL7,yL8,yL9,yL10,alpha,M,Re
214405,0.076103,0.132254,0.170253,0.185008,0.176889,0.150714,0.113887,0.074184,0.038813,0.014231,...,-0.164893,-0.154471,-0.128776,-0.095385,-0.060752,-0.030424,-0.009414,6.0,0.1,500000
96699,0.037953,0.070476,0.095600,0.106901,0.102431,0.085378,0.062389,0.039328,0.019651,0.006310,...,-0.043063,-0.039651,-0.034625,-0.027477,-0.018611,-0.010077,-0.003843,5.0,0.3,500000
25137,0.029451,0.053592,0.069208,0.074846,0.070859,0.059739,0.044699,0.028791,0.014739,0.004992,...,-0.075095,-0.071156,-0.060050,-0.044992,-0.029044,-0.014950,-0.005170,-10.0,0.3,300000
216989,0.091555,0.110015,0.080970,0.040807,-0.001348,-0.034089,-0.048429,-0.042975,-0.025120,-0.006870,...,-0.011791,-0.050275,-0.074383,-0.078339,-0.062550,-0.035451,-0.010515,7.0,0.3,300000
33115,0.009768,0.017685,0.022712,0.024403,0.022916,0.019104,0.014044,0.008740,0.004080,0.000856,...,-0.025577,-0.024422,-0.020825,-0.015854,-0.010539,-0.005817,-0.002531,9.0,0.1,200000
15400,0.058951,0.107285,0.138594,0.149941,0.142015,0.119788,0.089691,0.057835,0.029690,0.010163,...,-0.149941,-0.142015,-0.119788,-0.089691,-0.057835,-0.029690,-0.010163,-3.0,0.3,400000
165156,0.069186,0.123521,0.168393,0.191484,0.186132,0.154261,0.106518,0.058249,0.023817,0.006741,...,-0.008857,-0.004090,-0.008528,-0.016517,-0.020693,-0.016197,-0.006838,2.0,0.3,500000
96343,0.026112,0.051479,0.072144,0.081914,0.078676,0.065176,0.047218,0.029528,0.014618,0.004593,...,-0.018062,-0.016029,-0.014802,-0.012673,-0.009087,-0.005198,-0.002180,6.0,0.3,300000
74373,0.071728,0.122151,0.157238,0.171224,0.163230,0.137509,0.102096,0.065300,0.033172,0.011061,...,-0.128681,-0.120935,-0.102368,-0.077534,-0.050510,-0.026238,-0.009215,2.0,0.1,200000
104448,0.071394,0.127794,0.164888,0.178705,0.169886,0.143862,0.108104,0.069986,0.036201,0.012746,...,-0.171162,-0.161479,-0.135635,-0.101165,-0.064948,-0.033054,-0.010938,5.0,0.1,400000


In [16]:
# displaying the targets table for NACA4D_10
y4.head(10)

,CL,CD,Cm
19046,1.480,0.03675,-0.025
116332,1.393,0.03216,-0.193
93273,0.872,0.02125,-0.106
64947,1.134,0.02502,-0.119
162156,1.771,0.03397,-0.136
166348,0.836,0.02459,-0.223
50236,-0.445,0.02950,-0.081
137968,2.411,0.03913,-0.319
174770,1.552,0.02653,-0.480
94278,0.581,0.02615,-0.095


In [17]:
# displaying the targets table for NACA5D_10
y5.head(10)

,CL,CD,Cm
214405,1.008,0.03264,-0.041
96699,0.989,0.02036,-0.060
25137,-1.290,0.02746,0.012
216989,0.005,0.03924,0.388
33115,1.027,0.01947,-0.003
15400,-0.434,0.02738,0.007
165156,0.976,0.02625,-0.071
96343,1.068,0.01988,-0.060
74373,0.539,0.02916,-0.037
104448,0.772,0.03183,-0.022


## Splitting the Dataset

In [18]:
# assigning X_train, y_train, X_test & y_test
X_train, X_val, y_train, y_val = train_test_split(X5, y5, train_size=0.8, random_state=42)
X_test, y_test = X4, y4

In [19]:
# normalizing the training, validation & testing datasets
X_scaler = StandardScaler()
X_train = X_scaler.fit_transform(X_train)
X_val = X_scaler.transform(X_val)
X_test = X_scaler.transform(X_test)

In [20]:
# displaying X_train, X_val & X_test after standardization
display(pd.DataFrame(X_train).head())
display(pd.DataFrame(X_val).head())
display(pd.DataFrame(X_test).head())

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,-0.012704,0.294102,0.393273,0.440523,0.483155,0.547253,0.636264,0.743727,0.859354,1.016281,...,-1.812375,-1.776110,-1.724646,-1.645291,-1.539676,-1.437596,-1.440657,-1.650779,-0.001482,-0.706701
1,1.044904,0.644789,0.490676,0.372053,0.214937,0.026000,-0.169960,-0.330230,-0.409504,-0.272994,...,-1.156827,-1.370607,-1.588192,-1.799806,-1.974870,-2.063237,-1.989466,0.660369,-0.001482,1.412832
2,-0.855146,-0.878225,-0.822311,-0.731059,-0.628363,-0.519867,-0.406757,-0.289030,-0.155312,0.108918,...,0.014543,0.133646,0.258277,0.385012,0.510437,0.647885,0.948966,0.990533,-0.001482,0.706321
3,-0.299907,-0.620262,-0.796920,-0.837948,-0.819515,-0.770761,-0.706493,-0.640234,-0.596940,-0.675694,...,0.658843,0.697359,0.765350,0.845769,0.920709,0.974143,0.994310,-0.495205,1.223435,1.412832
4,0.214439,0.476136,0.563103,0.624434,0.693748,0.783433,0.888996,1.004050,1.120768,1.287468,...,-1.625371,-1.557582,-1.476119,-1.378819,-1.269535,-1.175639,-1.188780,-1.650779,-0.001482,-1.413212


,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,1.957842,1.795356,1.635188,1.558655,1.552976,1.592139,1.658997,1.738997,1.802666,1.809594,...,-0.680519,-0.750035,-0.750856,-0.713860,-0.660199,-0.628849,-0.756662,-1.485697,-0.001482,-0.706701
1,-0.055041,0.226311,0.406134,0.427903,0.288503,0.087971,-0.110830,-0.295432,-0.444976,-0.563344,...,1.184337,1.041319,0.853401,0.663020,0.486403,0.345648,0.279422,-0.330123,1.223435,0.706321
2,1.391450,0.704165,0.156181,-0.261584,-0.659035,-1.069434,-1.463436,-1.778040,-1.939161,-1.740286,...,-0.350904,-0.750060,-1.151023,-1.544702,-1.874595,-2.047429,-1.843010,-1.650779,1.223435,0.706321
3,0.010172,0.179485,0.256248,0.327685,0.412303,0.514793,0.630169,0.755191,0.893987,1.158599,...,-1.172806,-1.083764,-0.982242,-0.869190,-0.747121,-0.625360,-0.485392,1.155615,1.223435,-1.413212
4,-1.470987,-1.681431,-1.689224,-1.624090,-1.541305,-1.455928,-1.368861,-1.287555,-1.231079,-1.295879,...,0.616583,0.717701,0.826340,0.936594,1.036324,1.117555,1.211204,0.825451,1.223435,-0.706701


,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,0.016421,-0.088213,-0.050192,0.001809,0.070070,0.158177,0.258228,0.361712,0.450331,0.496341,...,-0.749574,-0.679784,-0.591854,-0.490926,-0.387965,-0.308515,-0.325817,1.650861,-0.001482,-0.706701
1,0.604991,0.723114,0.850418,1.021959,1.234445,1.470488,1.720736,1.957748,2.111604,2.028105,...,-0.482695,-0.236305,0.011724,0.251527,0.441770,0.530867,0.334014,0.495287,-1.226398,1.412832
2,-0.478634,-0.344792,-0.199261,-0.139568,-0.065444,0.046101,0.169892,0.284023,0.348144,0.224336,...,0.614413,0.715649,0.847687,0.982536,1.090397,1.141099,1.052761,0.330205,-0.001482,-0.706701
3,-0.458478,-0.401181,-0.296202,-0.159403,0.007719,0.204648,0.410835,0.594019,0.714240,0.649450,...,-0.162866,0.027248,0.244861,0.462525,0.638007,0.737096,0.655660,0.825451,-1.226398,-0.000190
4,0.791656,0.970931,0.858135,0.794512,0.840757,0.952789,1.088836,1.217601,1.279613,1.101071,...,0.808247,0.871636,0.991979,1.122187,1.219833,1.242263,1.045583,0.825451,-0.001482,-0.706701


In [21]:
# displaying y_train, y_val & y_test after standardization
display(pd.DataFrame(y_train).head())
display(pd.DataFrame(y_val).head())
display(pd.DataFrame(y_test).head())

,CL,CD,Cm
79191,-1.632,0.04615,0.053
108353,0.096,0.01735,0.201
213229,0.877,0.01996,-0.034
127141,-0.102,0.02227,-0.008
57120,-1.431,0.04717,0.021


,CL,CD,Cm
154897,-0.577,0.02127,-0.011
138356,0.182,0.01613,0.012
173943,-2.133,0.01569,0.331
148172,1.101,0.03827,-0.033
246,0.597,0.01484,-0.003


,CL,CD,Cm
19046,1.480,0.03675,-0.025
116332,1.393,0.03216,-0.193
93273,0.872,0.02125,-0.106
64947,1.134,0.02502,-0.119
162156,1.771,0.03397,-0.136


In [22]:
# defining the learning rate reduction callback
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=5, min_lr=0.00001)

### Model [512, 256, 128, 3]

In [23]:
# RMSE and R² metric scores on test data for Model M_512_256_128
M_512_256_128_metrics = []

for i in range(20):
    print(f"\nRun {i+1}/20 for Model M_512_256_128...")

    M_512_256_128 = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(512, activation='relu'),
        Dense(256, activation='relu'),
        Dense(128, activation='relu'),
        Dense(y_train.shape[1], activation='linear')
    ])

    M_512_256_128.compile(
        optimizer=Adam(learning_rate=5e-4, beta_1=0.9, beta_2=0.999),
        loss='mse',
        metrics=[RootMeanSquaredError()]
    )

    M_512_256_128.fit(
        X_train, y_train,
        epochs=50, batch_size=128,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr],
        verbose=0
    )

    y_pred = M_512_256_128.predict(X_test, verbose=0)
    r2 = r2_score(y_test, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))

    M_512_256_128_metrics.append({
        'CL_rmse': rmse[0], 'CD_rmse': rmse[1], 'Cm_rmse': rmse[2],
        'CL_r2': r2[0], 'CD_r2': r2[1], 'Cm_r2': r2[2]
    })

results_M_512_256_128_df = pd.DataFrame(M_512_256_128_metrics)

print(results_M_512_256_128_df.mean())



Run 1/20 for Model M_512_256_128...


I0000 00:00:1764217160.184620 4188066 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1183 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9
I0000 00:00:1764217163.079112 4189113 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.



Run 2/20 for Model M_512_256_128...

Run 3/20 for Model M_512_256_128...

Run 4/20 for Model M_512_256_128...

Run 5/20 for Model M_512_256_128...

Run 6/20 for Model M_512_256_128...

Run 7/20 for Model M_512_256_128...

Run 8/20 for Model M_512_256_128...

Run 9/20 for Model M_512_256_128...

Run 10/20 for Model M_512_256_128...

Run 11/20 for Model M_512_256_128...

Run 12/20 for Model M_512_256_128...

Run 13/20 for Model M_512_256_128...

Run 14/20 for Model M_512_256_128...

Run 15/20 for Model M_512_256_128...

Run 16/20 for Model M_512_256_128...

Run 17/20 for Model M_512_256_128...

Run 18/20 for Model M_512_256_128...

Run 19/20 for Model M_512_256_128...

Run 20/20 for Model M_512_256_128...
CL_rmse    0.026671
CD_rmse    0.005698
Cm_rmse    0.011083
CL_r2      0.999092
CD_r2      0.722853
Cm_r2      0.988516
dtype: float32
